# Case de Estudos: Santander Value Prediction

Ajude Santander a identificar o valor das transações para cada cliente potencial. Esse é um primeiro passo que o Santander precisa acertar para personalizar seus serviços em grande escala.
De acordo com uma pesquisa da Epsilon, 80% dos clientes tendem a voltar a fazer negócios com a sua empresa se a mesma entregar um serviço personalizado.

<br>
## Link para os dados e o desafio: 

https://www.kaggle.com/c/santander-value-prediction-challenge/data

O case podera ser quebrado nas 6 partes seguintes:

    Identificar o problema
        Qual o tipo de problema(classificação, regressão, clustering)?
    Necessidades de aplicar transformaçoes?
        Ex: imputing de valores null, encoding de colunas string, etc
    Separar os sets de treinamento e teste
    Baseline
        Achar uma baseline, um primeiro modelo para ter uma referencia
    Escolher a metrica
    Melhorar o resultado
        Feature engineering, otimizaçao do modelo, hiperparametros, etc

